In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from mlflow import MlflowClient
import mlflow 
from mlflow.models import infer_signature

In [2]:
mlflow.set_tracking_uri(uri="http://127.0.0.1:5000")

In [3]:
#bazowy model, bez feature engineeringu poza podstawowym one-hot encodingiem i bez fillowania na
data = pd.read_csv('../data/train.csv', index_col='PassengerId')
data = pd.concat([data[['Survived', 'Pclass', 'Age', 'Fare','SibSp', 'Parch']], pd.get_dummies(data[['Embarked','Sex']])],axis=1).dropna()
X = data.drop('Survived', axis=1)
y = data['Survived']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
params = {'C': 1.0, 'class_weight': None, 'dual': False, 'fit_intercept': True, 'intercept_scaling': 1, 'l1_ratio': None, 'max_iter': 100, 'multi_class': 'auto', 'n_jobs': None, 'penalty': 'l2', 'random_state': None, 'solver': 'lbfgs', 'tol': 0.0001, 'verbose': 0, 'warm_start': False}
lr = LogisticRegression(**params)
lr.fit(X_train, y_train)

y_pred = lr.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)   

c:\Users\pkwiecien\AppData\Local\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [4]:
experiment_description = (
    "test mlflow na bazie setu titanic"
)

experiment_tags = {
    "project_name": "titanic-mlflow-test",
    "owner": "pkwiecien",
    "mlflow.note.content": experiment_description,
}

mlflow.create_experiment(name="baseline", tags=experiment_tags)

mlflow.set_experiment("baseline")

run_name = "baseline_LR"

artifact_path = "LR1"

metrics = {
    "accuracy": accuracy,
}

signature = infer_signature(X_train, lr.predict(X_train))


with mlflow.start_run(run_name=run_name) as run:
    mlflow.log_params(params)
    mlflow.log_metrics(metrics)
    mlflow.sklearn.log_model(sk_model=lr, input_example=X_train, signature=signature, artifact_path=artifact_path,registered_model_name="lr1")

c:\Users\pkwiecien\AppData\Local\anaconda3\lib\site-packages\mlflow\models\signature.py:212: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details.
  inputs = _infer_schema(model_input) if model_input is not None else None
c:\Users\pkwiecien\AppData\Local\anaconda3\lib\site-packages\_distutils_hack\__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setupt